In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

headers = {
    "Referer": "https://int.soccerway.com/",
    "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}

# League URL

In [3]:
url_league_summary = {
    'https://int.soccerway.com/national/england/premier-league/': 'England Premier League',
    # 'https://int.soccerway.com/national/spain/primera-division/': 'Spain LaLiga',
    # 'https://int.soccerway.com/national/italy/serie-a/': 'Italy Serie A',
    # 'https://int.soccerway.com/national/germany/bundesliga/': 'Germany Bundesliga',
    # 'https://int.soccerway.com/national/france/ligue-1/': 'France Ligue 1',
}

url_league_sidelined = {
    'https://int.soccerway.com/national/england/premier-league/20242025/regular-season/r81780/sidelined/' : 'England Premier League',
    # 'https://int.soccerway.com/national/spain/primera-division/20242025/regular-season/r82318/sidelined/' : 'Spain LaLiga',
    # 'https://int.soccerway.com/national/italy/serie-a/20242025/regular-season/r82869/sidelined/' : 'Italy Serie A',
    # 'https://int.soccerway.com/national/germany/bundesliga/20242025/regular-season/r81840/sidelined/' : 'Germany Bundesliga',
    # 'https://int.soccerway.com/national/france/ligue-1/20242025/regular-season/r81802/sidelined/' : 'France Ligue 1',
}

# Get Team URL

In [53]:
# Hàm lấy dữ liệu từ một giải đấu
def get_league_teams(url, league_name):
    # Gửi yêu cầu HTTP
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Không thể truy cập vào url của giải đấu {league_name}")
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm bảng có class 'detailed-table'
    table = soup.find('table', class_='detailed-table')

    # Tìm tất cả các <td> chứa thông tin đội bóng
    teams_data = []
    for row in table.find_all('td', class_='text team large-link'):
        # Lấy tên đội từ thuộc tính title và URL của đội từ thuộc tính href
        team_name = row.find('a')['title']
        team_url = "https://int.soccerway.com" + row.find('a')['href']
        teams_data.append([league_name, url, team_name, team_url])

    return teams_data

# Duyệt qua các giải đấu và lấy dữ liệu cho từng giải
all_teams = []
for url, league_name in url_league_summary.items():
    league_teams = get_league_teams(url, league_name)
    print(f"{league_name} Get done...")
    all_teams.extend(league_teams)

# Tạo DataFrame từ dữ liệu
df_team = pd.DataFrame(all_teams, columns=['League', 'League URL', 'Team', 'Team URL'])

England Premier League Get done...


# Get Players URL

In [72]:
def get_squad_data(team_link, headers):
    squad_url = team_link + 'squad/'
    response = requests.get(squad_url, headers=headers)
    if response.status_code != 200:
        print(f"Không thể truy cập vào url của {squad_url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Tìm bảng có class "table squad sortable"
    squad_table = soup.find('table', class_='table squad sortable')
    
    # Lấy dữ liệu trong bảng, chỉ lấy các cột theo thứ tự yêu cầu
    rows = []
    for row in squad_table.find('tbody').find_all('tr'):
        # Lấy từng cột mong muốn từ bảng, ví dụ:
        shirtnum = row.find_all('td')[0].text.strip()  # Shirtnum
        player_img = row.find('img')['src']            # Player img
        player_url = "https://int.soccerway.com" + row.find('a')['href'] # Player URL
        player_id = player_url.split('/')[-2]
        player_name = row.find_all('td')[2].text.strip()  # Player name
        age = row.find_all('td')[4].text.strip()       # Age
        position = row.find_all('td')[5].text.strip()  # Position
        min_played = row.find_all('td')[6].text.strip()  # Minutes played
        appearance = row.find_all('td')[7].text.strip()  # Appearances
        lineups = row.find_all('td')[8].text.strip()   # Lineups
        subs_in = row.find_all('td')[9].text.strip()   # Subs In
        subs_out = row.find_all('td')[10].text.strip() # Subs Out
        subs_on_bench = row.find_all('td')[11].text.strip() # Subs on bench
        goals = row.find_all('td')[12].text.strip()    # Goals
        assists = row.find_all('td')[13].text.strip()  # Assists
        yellow_cards = row.find_all('td')[14].text.strip()  # YC
        yellow_red_cards = row.find_all('td')[15].text.strip()  # YRC
        red_cards = row.find_all('td')[16].text.strip()  # RC
        
        # Thêm tất cả dữ liệu vào hàng
        rows.append([shirtnum, player_img, player_url, player_id, player_name, age, position, min_played, 
                     appearance, lineups, subs_in, subs_out, subs_on_bench, goals, assists, 
                     yellow_cards, yellow_red_cards, red_cards])
    
    # Chuyển dữ liệu thành DataFrame
    df = pd.DataFrame(rows, columns=['Shirtnum', 'Player_img', 'Player_URL', 'Player_ID', 'Player', 'Age', 'Position', 
                                     'Min_Played', 'Appearance', 'Lineups', 'Subs_In', 'Subs_Out', 
                                     'Subs_on_bench', 'Goal', 'Assist', 'YC', 'YRC', 'RC'])
    
    return df

# Khởi tạo danh sách để lưu kết quả cho tất cả các đội
all_teams_squad_data = []

# Giả sử df_team đã có sẵn với cột 'Team Link'
# Lặp qua từng team link trong df_team
for index, row in df_team.iterrows():
    team_name = row['Team']        # Lấy tên đội bóng   row['Team']
    team_link = row['Team URL']    # Lấy link đội bóng   row['Team URL']
    
    print(f"Fetching squad data for {team_name}...")  # In thông báo
    
    # Lấy dữ liệu đội hình cho đội bóng đó
    squad_df = get_squad_data(team_link, headers=headers)
    
    # Thêm cột 'Team' vào DataFrame để biết đội bóng nào
    squad_df['Team'] = team_name
    
    # Lưu kết quả vào danh sách
    all_teams_squad_data.append(squad_df)

# Gộp tất cả DataFrame lại thành một DataFrame duy nhất
final_squad_df = pd.concat(all_teams_squad_data, ignore_index=True)

Fetching squad data for Manchester City...
Fetching squad data for Liverpool...
Fetching squad data for Aston Villa...
Fetching squad data for Arsenal...
Fetching squad data for Chelsea...
Fetching squad data for Newcastle United...
Fetching squad data for Brighton & Hove Albion...
Fetching squad data for Nottingham Forest...
Fetching squad data for Fulham...
Fetching squad data for Tottenham Hotspur...
Fetching squad data for Manchester United...
Fetching squad data for Brentford...
Fetching squad data for AFC Bournemouth...
Fetching squad data for West Ham United...
Fetching squad data for Leicester City...
Fetching squad data for Crystal Palace...
Fetching squad data for Ipswich Town...
Fetching squad data for Southampton...
Fetching squad data for Everton...
Fetching squad data for Wolverhampton Wanderers...


# Get Players Information

In [127]:
def get_player_info(player_url, headers):
    try:
        response = requests.get(player_url, headers=headers)
        response.raise_for_status()  # Kiểm tra lỗi HTTP
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Tìm thẻ chứa thông tin cầu thủ
        player_info_section = soup.find('div', class_='block_player_passport real-content clearfix')
        player_info = {}
        
        if player_info_section:
            # Tìm tất cả các cặp <dt> và <dd>
            details = player_info_section.find_all('dl')
            for detail in details:
                dt_tags = detail.find_all('dt')
                dd_tags = detail.find_all('dd')
                for dt, dd in zip(dt_tags, dd_tags):
                    label = dt.text.strip()
                    value = dd.text.strip()
                    player_info[label] = value

        # Lấy thông tin chuyển nhượng
        transfers_history = get_transfers_history(soup)
        injuried_suspension_history = get_injuried_suspension_history(soup)
        
        # Trả về các thông tin cần thiết
        return {
            'First name': player_info.get('First name', ''),
            'Last name': player_info.get('Last name', ''),
            'Nationality': player_info.get('Nationality', ''),
            'Date of birth': player_info.get('Date of birth', ''),
            'Age': player_info.get('Age', ''),
            'Country of birth': player_info.get('Country of birth', ''),
            'Place of birth': player_info.get('Place of birth', ''),
            'Position': player_info.get('Position', ''),
            'Height': player_info.get('Height', ''),
            'Weight': player_info.get('Weight', ''),
            'Foot': player_info.get('Foot', ''),
            'Transfers_History': transfers_history,
            'Injuried_Suspension_History': injuried_suspension_history,
        }
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred for {player_url}: {err}")
    except Exception as e:
        print(f"An error occurred for {player_url}: {e}")

def get_transfers_history(soup):
    transfers_history = []
    transfers_table = soup.find('table', class_='transfers table')
    
    if transfers_table:
        rows = transfers_table.find_all('tr')[1:]  # Bỏ qua hàng tiêu đề
        for row in rows:
            date = row.find('td', class_='date').text.strip()
            from_team = row.find_all('td', class_='team')[0].text.strip()
            to_team = row.find_all('td', class_='team')[1].text.strip()
            fee = row.find('td', class_='type').text.strip()
            transfer_info = f"{date} - From {from_team} - To {to_team} - Fee {fee}"
            transfers_history.append(transfer_info)

    return ', '.join(transfers_history)

def get_injuried_suspension_history(soup):
    history = []
    injuries_table = soup.find('table', class_='sidelined table')
    
    if injuries_table:
        rows = injuries_table.find_all('tr')[1:]  # Bỏ qua hàng tiêu đề
        for row in rows:
            description = row.find_all('td')[1].text.strip()  # Mô tả ở td thứ 2
            start_date_str = row.find_all('td')[2].text.strip()  # Ngày bắt đầu ở td thứ 3
            end_date_str = row.find_all('td')[3].text.strip()  # Ngày kết thúc ở td thứ 4
            
            # Tính số ngày nếu có dữ liệu ngày tháng
            if start_date_str and end_date_str:
                total_days = calculate_days(start_date_str, end_date_str)
                injury_info = (f"Description {description} - Start Date {start_date_str} "
                               f"- End Date {end_date_str} - Total Days {total_days}")
                history.append(injury_info)

    return ', '.join(history)

def calculate_days(start_date_str, end_date_str):
    try:
        # Chuyển đổi chuỗi ngày tháng thành đối tượng datetime
        start_date = datetime.strptime(start_date_str, '%d/%m/%y')
        end_date = datetime.strptime(end_date_str, '%d/%m/%y')
        total_days = (end_date - start_date).days + 1  # Cộng thêm 1 để tính cả ngày bắt đầu
        return total_days
    except ValueError as e:
        print(f"Error parsing dates: {e}")
        return 0

# Giả sử bạn đã có DataFrame `final_squad_df` chứa cột `Player URL`
player_info_list = []

# Lặp qua từng dòng của DataFrame
for index, row in final_squad_df.head(2).iterrows():
    player_url = row['Player_URL']
    
    print(f"Fetching data for player: {row['Player']} from {player_url}")
    
    # Lấy thông tin chi tiết cầu thủ từ URL
    player_info = get_player_info(player_url, headers=headers)
    time.sleep(2)  # Nghỉ 2 giây giữa các yêu cầu
    
    if player_info:
        player_info['Player_ID'] = row['Player_ID']  # Thêm Player ID vào dữ liệu
        player_info['Player'] = row['Player']
        player_info_list.append(player_info)

# Tạo DataFrame từ danh sách player_info_list
player_info_df = pd.DataFrame(player_info_list)



Fetching data for player: S. Ortega from https://int.soccerway.com/players/stefan-ortega-moreno/152598/
Fetching data for player: Ederson from https://int.soccerway.com/players/ederson--santana-de-moraes/206804/


In [128]:
display(player_info_df)

,First name,Last name,Nationality,Date of birth,Age,Country of birth,Place of birth,Position,Height,Weight,Foot,Transfers_History,Injuried_Suspension_History,Player_ID,Player
0,Stefan,Ortega Moreno,Germany,6 November 1992,31,Germany,Hofgeismar,Goalkeeper,185 cm,88 kg,Right,01/07/22 - From Bielefeld - To Man City - Fee ...,,152598,S. Ortega
1,Ederson,Santana de Moraes,Brazil,17 August 1993,31,Brazil,Osasco,Goalkeeper,188 cm,86 kg,Left,01/07/17 - From Benfica - To Man City - Fee € ...,Description Eye Injury - Start Date 15/05/24 -...,206804,Ederson


In [121]:
# Lưu DataFrame vào tệp CSV
player_info_df.to_csv('player_info.csv', index=False)

# Get Team Players Injuried & Suspension

In [4]:
url_league_sidelined = {
    'https://int.soccerway.com/national/england/premier-league/20242025/regular-season/r81780/sidelined/' : 'England Premier League',
    # 'https://int.soccerway.com/national/spain/primera-division/20242025/regular-season/r82318/sidelined/' : 'Spain LaLiga',
    # 'https://int.soccerway.com/national/italy/serie-a/20242025/regular-season/r82869/sidelined/' : 'Italy Serie A',
    # 'https://int.soccerway.com/national/germany/bundesliga/20242025/regular-season/r81840/sidelined/' : 'Germany Bundesliga',
    # 'https://int.soccerway.com/national/france/ligue-1/20242025/regular-season/r81802/sidelined/' : 'France Ligue 1',
}

# Danh sách để lưu dữ liệu
data = []

# Duyệt qua các URL và lấy thông tin bảng
for url, league_name in url_league_sidelined.items():
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        print(f"Truy cập thành công trang: {league_name}")

        # Phân tích HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm tất cả các nhóm trong bảng
        teams = soup.find_all('tr', class_='group-head')
        
        for team in teams:
            team_name = team.get_text(strip=True)  # Tên đội bóng
            
            # Tìm hàng tiếp theo cho cầu thủ
            next_row = team.find_next_sibling('tr')
            while next_row and 'sub-head' not in next_row['class']:
                next_row = next_row.find_next_sibling('tr')  # Tiến đến hàng có tiêu đề cầu thủ
            
            # Duyệt qua các cầu thủ trong nhóm
            while next_row and 'group-head' not in next_row['class']:
                cols = next_row.find_all('td')
                if cols:
                    player_link = cols[0].find('a')['href']  # Link đến cầu thủ
                    player_name = cols[0].get_text(strip=True)  # Tên cầu thủ
                    injury_type = cols[1]['title'] if len(cols) > 1 else None  # Loại chấn thương
                    start_date = cols[2].get_text(strip=True)  # Ngày bắt đầu
                    end_date = cols[3].get_text(strip=True)  # Ngày kết thúc
                    
                    # Thêm thông tin vào danh sách
                    data.append([team_name, player_name, player_link, injury_type, start_date, end_date])
                
                # Tiến đến hàng tiếp theo
                next_row = next_row.find_next_sibling('tr')
    else:
        print(f"Không thể truy cập {url}, mã lỗi: {response.status_code}")

# Tạo DataFrame từ danh sách
df_sidelined = pd.DataFrame(data, columns=['Team', 'Player', 'Player link', 'Injury Type', 'Start Date', 'End Date'])

Truy cập thành công trang: England Premier League


In [5]:
display(df_sidelined)

,Team,Player,Player link,Injury Type,Start Date,End Date
0,AFC Bournemouth,T. Adams,/players/tyler-adams/400878/,Back Injury,20/07/24,20/10/24
1,Arsenal,M. Ødegaard,/players/martin-odegaard/334861/,Ankle Injury,10/09/24,20/10/24
2,Arsenal,O. Zinchenko,/players/oleksandr-zinchenko/275836/,Calf Injury,11/09/24,20/10/24
3,Arsenal,K. Tierney,/players/kieran-tierney/323402/,Hamstring,20/06/24,
4,Arsenal,B. White,/players/ben-white/452343/,Groin Injury,23/09/24,
...,...,...,...,...,...,...
83,Wolverhampton Wanderers,B. Traoré,/players/boubacar-traore/597538/,Knee Injury,29/08/24,27/12/24
84,Wolverhampton Wanderers,Y. Mosquera,/players/yerson-mosquera/648500/,ACL Knee Injury,22/09/24,01/06/25
85,Wolverhampton Wanderers,S. Kalajdžić,/players/sasa-kalajdzic/349011/,LCL Knee Injury,19/02/24,
86,Wolverhampton Wanderers,E. González,/players/enso-david-gonzalez-medina/864258/,Knee Injury,28/07/24,
